In [1]:
import tensorflow as tf

print('\u2022 Using TensorFlow Version:', tf.__version__)

• Using TensorFlow Version: 2.12.0


In [2]:
import os
import tensorflow as tf
os.listdir('Fruits')

['FreshApple',
 'FreshBanana',
 'FreshMango',
 'FreshOrange',
 'FreshStrawberry',
 'RottenApple',
 'RottenBanana',
 'RottenMango',
 'RottenOrange',
 'RottenStrawberry']

In [3]:
base_dir ='Fruits'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [4]:
fresh_apple_dir = os.path.join(base_dir, 'FreshApple')
fresh_banana_dir = os.path.join(base_dir, 'FreshBanana')
fresh_mango_dir = os.path.join(base_dir, 'FreshMango')
fresh_orange_dir = os.path.join(base_dir, 'FreshOrange')
fresh_strawberry_dir = os.path.join(base_dir, 'FreshStrawberry')
rotten_apple_dir = os.path.join(base_dir, 'RottenApple')
rotten_banana_dir = os.path.join(base_dir, 'RottenBanana')
rotten_mango_dir = os.path.join(base_dir, 'RottenMango')
rotten_orange_dir = os.path.join(base_dir, 'RottenOrange')
rotten_strawberry_dir = os.path.join(base_dir, 'RottenStrawberry')

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1.0/255,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip= True,
                    fill_mode='nearest',
                    validation_split=0.2
                    )

train_generator = train_datagen.flow_from_directory(
                      base_dir, 
                      target_size= (150, 150),  
                      batch_size= 32,
                      shuffle = True,
                      subset = 'training',
                      class_mode= 'categorical' 
                      )


validation_generator = train_datagen.flow_from_directory(
                          base_dir, 
                          target_size= (150, 150),
                          batch_size= 32, 
                          shuffle = True,
                          subset = 'validation',
                          class_mode= 'categorical'
                          )

Found 4795 images belonging to 10 classes.
Found 1193 images belonging to 10 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Activation, Flatten, MaxPool2D
from tensorflow.keras.applications import ResNet50


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(1024, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 512)       590336    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 512)      0

In [7]:
model.compile(loss = 'categorical_crossentropy',
              optimizer= tf.optimizers.Adam(), 
              metrics= ['accuracy'])


In [8]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > 0.95 and logs.get('val_accuracy') > 0.95):
            print("\nakurasi lebih besar dari 96%. maka training berhenti!") 
            self.model.stop_training = True 
 
callbacks = myCallback()

In [ ]:
history = model.fit(
          train_generator,
          epochs=30, 
          steps_per_epoch=len(train_generator),
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          verbose=2,
          callbacks = [callbacks])

Epoch 1/30
150/150 - 670s - loss: 2.2018 - accuracy: 0.1894 - val_loss: 2.0098 - val_accuracy: 0.2967 - 670s/epoch - 4s/step
Epoch 2/30
150/150 - 633s - loss: 1.7795 - accuracy: 0.3689 - val_loss: 1.6840 - val_accuracy: 0.3738 - 633s/epoch - 4s/step
Epoch 3/30
150/150 - 705s - loss: 1.5244 - accuracy: 0.4688 - val_loss: 1.4643 - val_accuracy: 0.4786 - 705s/epoch - 5s/step
Epoch 4/30
150/150 - 749s - loss: 1.2960 - accuracy: 0.5447 - val_loss: 1.3648 - val_accuracy: 0.5423 - 749s/epoch - 5s/step
Epoch 5/30
150/150 - 717s - loss: 1.1174 - accuracy: 0.6190 - val_loss: 1.2993 - val_accuracy: 0.5390 - 717s/epoch - 5s/step
Epoch 6/30
150/150 - 710s - loss: 0.9826 - accuracy: 0.6636 - val_loss: 1.2610 - val_accuracy: 0.5432 - 710s/epoch - 5s/step
Epoch 7/30
150/150 - 710s - loss: 0.8317 - accuracy: 0.7233 - val_loss: 0.9103 - val_accuracy: 0.6773 - 710s/epoch - 5s/step
Epoch 8/30
150/150 - 672s - loss: 0.7357 - accuracy: 0.7593 - val_loss: 0.7787 - val_accuracy: 0.7276 - 672s/epoch - 4s/step


In [ ]:
loss = model.evaluate(validation_generator, steps=len(validation_generator))